In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("CustomerFileLoad").getOrCreate()

# Load customer file (assuming it's a CSV)
customer_df = spark.read.csv("Customer_table.csv", header=True, inferSchema=True)
# Show a sample of the data
customer_df.show()

+---------------+---------------+-----------+-------------------+-------------+-------+-----------------+---------------+-------------+---------+------------+---------------+--------------+----------------------+---------+----------------+------------+--------------+----------------+--------+-----------+
|Pickup DateTime|  Drop DateTime|Driver Name|Driver Phone Number|Trip Distance|Trip ID|  Pickup Location|  Drop Location|Trip Duration|Trip Fare|Vehicle Type|   Trip Purpose|Passenger Name|Passenger Phone Number|Scheduled|Passenger Rating|Driver Rated|Payment Method|Payment Discount|Net Fare|Trip Status|
+---------------+---------------+-----------+-------------------+-------------+-------+-----------------+---------------+-------------+---------+------------+---------------+--------------+----------------------+---------+----------------+------------+--------------+----------------+--------+-----------+
| 1/1/2016 21:11| 1/1/2016 21:17|     Almire|         9298608912|           21| 31

In [3]:
Admin_df = spark.read.csv("Admin_data.csv", header=True, inferSchema=True)
# Show a sample of the data
Admin_df.show()

+---------------+---------------+--------+----------+---+---------+-----------------+-----------+---------------+-----+-----------+-------------+---------------+-----+------+--------+--------+-----+-------------+---------+---------+---------+----------+----------+-----------+--------------+---------------+---------+----------+------------+--------+----------+---------+
|     Start_time|       End_time|    Name|    Mobile|Age|Pin-Codes|           Source|Vaccine_cus|    Destination|Miles|Est_Costing|Ride_category|        Purpose| temp|clouds|pressure|humidity| wind|accquire_vehi|free_vehi|Lattitute|Longitude|locationID|rating_cus|Riders_Name|Riders_contact|Trusted_Contact|Rating_RI|Vaccine_Ri|Payment_mode|Discount|Final_cost|   Status|
+---------------+---------------+--------+----------+---+---------+-----------------+-----------+---------------+-----+-----------+-------------+---------------+-----+------+--------+--------+-----+-------------+---------+---------+---------+----------+---

1.How many no of customers take trip from
same location.

In [6]:
from pyspark.sql.functions import count
same_triploc = customer_df.groupBy("Pickup Location").agg(count("Passenger Name").alias("customer_count"))
same_triploc.show()

+-----------------+--------------+
|  Pickup Location|customer_count|
+-----------------+--------------+
|      Fort Pierce|           108|
|  Lower Manhattan|            26|
|          Midtown|            78|
|     Midtown East|            26|
|Flatiron District|            26|
|      East Harlem|            26|
|    Hudson Square|            26|
|  West Palm Beach|            54|
|          Jamaica|            26|
|             Cary|            52|
|         New York|            26|
|         Elmhurst|            26|
+-----------------+--------------+



2.what is priority for each ride category from
each location.

In [19]:
from pyspark.sql.functions import count, col
result = Admin_df.groupBy("Source", "Ride_category").agg(count("*").alias("category_count"))
max_result = result.orderBy(col("Source"), col("category_count").desc())
final_df = max_result.dropDuplicates(["Source"])
final_df.show()
# max_result.show()







+-----------------+-------------+--------------+
|           Source|Ride_category|category_count|
+-----------------+-------------+--------------+
|             Cary|        Prime|            12|
|      East Harlem|         Bike|             6|
|         Elmhurst|         Auto|             6|
|Flatiron District|    Uber-Mini|             6|
|      Fort Pierce|         Bike|            25|
|    Hudson Square|        Prime|             6|
|          Jamaica|        Prime|             6|
|  Lower Manhattan|         Auto|             6|
|          Midtown|    Uber-Mini|            17|
|     Midtown East|        Prime|             6|
|         New York|         Auto|             6|
|  West Palm Beach|         Bike|            13|
+-----------------+-------------+--------------+



3.what are the longest locations of customer
travelled.

In [11]:
from pyspark.sql.functions import col
longesttrip_df = customer_df.orderBy(col("Trip Distance").desc()).select("Passenger Name", "Pickup Location", "Drop Location","Trip Distance").limit(10)
longesttrip_df.show()

+--------------+---------------+-------------+-------------+
|Passenger Name|Pickup Location|Drop Location|Trip Distance|
+--------------+---------------+-------------+-------------+
|      Virginie|       Elmhurst|         Cary|           80|
|      Charlena|   Midtown East|       Durham|           80|
|         Price|    East Harlem|  Whitebridge|           80|
|      Darlleen|    Fort Pierce|         Cary|           80|
|        Haskel|           Cary|  Whitebridge|           80|
|         Jacky|West Palm Beach|      Houston|           80|
|      Collette|    Fort Pierce|   Tanglewood|           80|
|          Moss|Lower Manhattan|  Morrisville|           80|
|          Elsy|       New York|         Cary|           79|
|         Daron|    Fort Pierce|         Cary|           79|
+--------------+---------------+-------------+-------------+



4. Drivers who completed ride with non
-vaccinated customers.

In [4]:
completed_rides_df = Admin_df.filter(Admin_df["Status"] == "Arrived")
nonvccus_df = completed_rides_df.filter(Admin_df["Vaccine_cus"] == "NO").select("Name")
nonvccus_df.show()


+--------+
|    Name|
+--------+
| Mathian|
|  Horten|
|   Boyce|
|    Berk|
|  Aubert|
| Stavros|
|  Norene|
|  Stefan|
| Cassius|
|   Chadd|
|  Joelle|
|   Abbey|
|  Kristo|
|   Lesli|
|    Marv|
| Sherwin|
|    Andi|
|   Adamo|
|Ernaline|
| Balduin|
+--------+
only showing top 20 rows



5.How many vaccinated customers have
travelled.

In [31]:
from pyspark.sql.functions import count
countcdv_df = Admin_df.filter(Admin_df["Vaccine_cus"] == "YES").agg(count("Riders_Name").alias("vaccinated_customer_count"))
countcdv_df.show()


+-------------------------+
|vaccinated_customer_count|
+-------------------------+
|                      400|
+-------------------------+



6.Customers who completed ride with non
-vaccinated Drivers.

In [5]:
completed_rides_df = Admin_df.filter(Admin_df["Status"] == "Arrived")
nonvaccinerd_df = completed_rides_df.filter(Admin_df["Vaccine_ri"] == "NO").select("Riders_Name")
nonvaccinerd_df.show()

+-----------+
|Riders_Name|
+-----------+
|     Mikkel|
|        Kip|
|      Cindi|
|    Yasmeen|
|       Burk|
|      Natty|
|       Kain|
|      Sandy|
|   Kimberly|
|    Carolus|
|       Dora|
|    Romonda|
|    Ellette|
|     Mikkel|
|       Abbi|
|     Pattin|
|     Haskel|
|      Cindi|
|    Yasmeen|
|       Burk|
+-----------+
only showing top 20 rows



7.who is the customer completed highest no
of rides.

In [24]:
from pyspark.sql.functions import count, max
trip_count_df = customer_df.groupBy("Passenger Name").agg(count("Trip ID").alias("trip_count"))
max_trip_count = trip_count_df.agg(max("trip_count")).collect()[0][0]
max_trip_customer_df = trip_count_df.filter(trip_count_df["trip_count"] == max_trip_count)
max_trip_customer_df.show()

+--------------+----------+
|Passenger Name|trip_count|
+--------------+----------+
|          Cher|        10|
+--------------+----------+



8.who is the driver completed highest no of
rides.

In [29]:
from pyspark.sql.functions import count, max
trip_count_df = Admin_df.groupBy("Name").agg(count("Name").alias("trip_count"))
max_trip_count = trip_count_df.agg(max("trip_count")).collect()[0][0]
max_trip_customer_df = trip_count_df.filter(trip_count_df["trip_count"] == max_trip_count)
max_trip_customer_df.show()

+---------+----------+
|     Name|trip_count|
+---------+----------+
|  Mathian|         2|
|   Teresa|         2|
|    Mayne|         2|
|Westbrook|         2|
|    Debor|         2|
|   Raeann|         2|
|      Gus|         2|
|  Othello|         2|
|  Frazier|         2|
| Roderigo|         2|
|   Bidget|         2|
|  Lisetta|         2|
+---------+----------+



9.what are first 10 age groups which uses
uber services mostly.

In [9]:
from pyspark.sql.functions import count, col
result = Admin_df.groupBy("Age").agg(count("Riders_Name").alias("customer_agecount"))
result = result.orderBy(col("customer_agecount").desc()).limit(10)
result.show()

+---+-----------------+
|Age|customer_agecount|
+---+-----------------+
| 69|               16|
| 15|               15|
| 32|               15|
| 28|               12|
| 78|               11|
| 43|               11|
| 49|               11|
| 21|               11|
| 23|               10|
| 44|               10|
+---+-----------------+



10.what is the count of different destination locations from same start location and also completed ride

In [42]:
from pyspark.sql.functions import  countDistinct, col
completed_rides_df = Admin_df.filter(Admin_df["Status"] == "Arrived")
result = completed_rides_df.groupBy("Source").agg(countDistinct("Destination").alias("distinct_destinations"))
result.show()


+-----------------+---------------------+
|           Source|distinct_destinations|
+-----------------+---------------------+
|      Fort Pierce|                   31|
|  Lower Manhattan|                    7|
|          Midtown|                   19|
|     Midtown East|                   11|
|Flatiron District|                    9|
|      East Harlem|                    8|
|    Hudson Square|                    7|
|  West Palm Beach|                   23|
|          Jamaica|                    8|
|             Cary|                   15|
|         New York|                    8|
|         Elmhurst|                    8|
+-----------------+---------------------+



Find the most expensive trip

In [4]:
from pyspark.sql.functions import max
Admin_df.filter(Admin_df["Final_cost"] == Admin_df.agg(max("Final_cost")).collect()[0][0]).select("Name", "Source", "Destination","Final_cost").show()



+----+------+-----------+----------+
|Name|Source|Destination|Final_cost|
+----+------+-----------+----------+
| Ely|  Cary|       Cary|    5406.0|
+----+------+-----------+----------+

